In [7]:
import imp
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path
from tqdm import tqdm

In [86]:
data_folder = "final_experiment/"
stem = "test_graphs/"
plot_folders = [stem+"compas/", stem+"compas_synth/", stem+"simple_synth/", stem+"lambda_test/"]
#sns.set(rc={'figure.figsize':(12,10)})
sns.set_style(style="darkgrid")
font = {'family': 'normal',
        'weight': 'bold',
        'size': 25}

plt.rc('font', **font)
#missing_pct = [0, 10,25,50,75, 90]
missing_pct = [0,
               50]
NAME_KEYS = {"coldel": "Column deletion",
             "cca": "CCA",
             "mice_def": "Chained Equation",
             "mean": "Mean",
             "log_reg": "Logistic Regression",
             "svm": "SVM",
             "knn": "KNN",
             "tnrd": "True Negative Ratio Difference",
             "tprd": "True Positive Ratio Difference",
             "rf_cat": "Random Forest",
             "eosum": "Sum of Equalised Odds",
             "spd": "Statistical Parity Difference",
             "acc": "Accuracy",
             "fair_reg_99": "Fair imputation lambda = 0.99",
             "fair_reg_95": "Fair imputation lambda = 0.95",
             "fair_reg_75": "Fair imputation lambda = 0.75",
             "fair_reg_15": "Fair imputation lambda = 0.15",
             "fair_reg_30": "Fair imputation lambda = 0.30",
             "fair_reg_45": "Fair imputation lambda = 0.45",
             "fair_reg_60": "Fair imputation lambda = 0.60",
             "fair_reg_75": "Fair imputation lambda = 0.75",
             "fair_reg_90": "Fair imputation lambda = 0.90"}

In [59]:
def load(name):
    with open(Path(data_folder+name), 'r') as f:
        return json.load(f)


In [60]:
#compas_priors = load("compas_priors.json")
compas = load("compas50_run_average.json")
synth = load("synth_compas50_run_average.json")
simple = load("simple_50_run_average.json")
lam = load("lambda_test.json")

In [61]:
compas['robust_Truepriors_count|is_Caucasian'].keys()

dict_keys(['Full data', 'Averaged results', 'Standard deviation'])

In [70]:
["fair_reg_"+str(a) for a in range(15,100,15)]

['fair_reg_15',
 'fair_reg_30',
 'fair_reg_45',
 'fair_reg_60',
 'fair_reg_75',
 'fair_reg_90']

In [63]:
IMPUTATIONS = [ "fair_reg_50", "log_reg", "cca", "mean", "mice_def", "coldel"]
lambda_imps =  ["fair_reg_"+str(a) for a in range(15,100,15)]

In [64]:
sns.load_dataset("tips")

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [90]:
lam['robust_Truepriors_count|gender_factor']['Averaged results']['mcar|spd|log_reg|fair_reg_15']

[0.3446395823284709, 0.30302617578912006]

In [85]:
er = "abcd"
print(er[-2:])

cd


In [98]:
def lam_plot(df, params, folder_path, imps):
    lambdas = [int(lam[-2:])/100 for lam in imps]
    savepath = folder_path+"/"
    if not os.path.isdir(Path(savepath)):
        os.mkdir(Path(savepath))
        
    for miss in tqdm(['mar', 'mcar']):
        for model in ["log_reg", "knn", "rf_cat"]:
            for metric in ["eosum", "acc", "tnrd", "tprd"]:
                full_data = {"Lambdas": lambdas,
                            NAME_KEYS[metric]: []}
                columns = ["Lambdas", NAME_KEYS[metric]]
                for imp in imps:
                    key = "|".join([miss,metric,model,imp])
                    data = df[params]["Averaged results"][key]
                    full_data[NAME_KEYS[metric]]+=[data[-1]]
                    #full_data["Missing percentage"].append([NAME_KEYS[imp]]*len(data))
                #print(full_data)
                plotting_data = pd.DataFrame(full_data, columns = columns)
                #print(plotting_data.head())
                fig = plt.gcf()
                fig.set_size_inches(30, 17.5)
                sns.lineplot(x="Lambdas", y=NAME_KEYS[metric], data = plotting_data)
                #fig.legend(bbox_to_anchor=(1.2,1))
                plt.title("Performance of "+NAME_KEYS[model]+" on the COMPAS dataset with " + miss)
                plt.show()
                """plt.savefig(Path(savepath+miss+"_"+model+"_"+metric+".png"))
                plt.clf() """  

In [108]:
a = pd.DataFrame({"a":[1,2,3,3], "b": [np.nan, 1, np.nan,3], "c": [1,0,1,1]}, columns=["a","b","c"])
list(a[a["c"]==1].index)

[0, 2, 3]

In [121]:
a = pd.DataFrame({"a":[1,2,3,3], "b": [np.nan, 1, np.nan,3], "c": [1,0,1,1]}, columns=["a","b","c"])
miss_r = pd.DataFrame([a.loc[a[a["c"]==1].index[0],:],a.loc[a[a["c"]==0].index[0],:]])
print(miss_r)
a = a.dropna()
a = pd.concat([a,miss_r], axis = 0, ignore_index =  True)
print(a)

     a    b    c
0  1.0  NaN  1.0
1  2.0  1.0  0.0
     a    b    c
0  2.0  1.0  0.0
1  3.0  3.0  1.0
2  1.0  NaN  1.0
3  2.0  1.0  0.0


In [ ]:
for key in lam.keys():
    print(key)
    lam_plot(lam, key, plot_folders[3]+"-".join(key.split("|")), lambda_imps)

In [66]:
def plotting(df, params, folder_path, imps = IMPUTATIONS):
    savepath = folder_path+"/"
    if not os.path.isdir(Path(savepath)):
        os.mkdir(Path(savepath))    
    for miss in tqdm(['mar', 'mcar']):
        for model in ["log_reg", "knn", "rf_cat"]:
            for metric in ["eosum", "acc", "spd"]:
                full_data = {"Missing percentage": missing_pct*len(imps),
                            NAME_KEYS[metric]: [],
                            "Imputation method": []}
                columns = ["Missing percentage", NAME_KEYS[metric], "Imputation method"]
                for imp in imps:
                    key = "|".join([miss,metric,model,imp])
                    data = df[params]["Averaged results"][key]
                    full_data["Imputation method"]+=[NAME_KEYS[imp]]*len(data)
                    full_data[NAME_KEYS[metric]]+=data
                    #full_data["Missing percentage"].append([NAME_KEYS[imp]]*len(data))
                #print(full_data)
                plotting_data = pd.DataFrame(full_data, columns = columns)
                #print(plotting_data.head())
                fig = plt.gcf()
                fig.set_size_inches(30, 17.5)
                sns.lineplot(x="Missing percentage", y=NAME_KEYS[metric], hue = "Imputation method", data = plotting_data)
                #fig.legend(bbox_to_anchor=(1.2,1))
                plt.title("Performance of "+NAME_KEYS[model]+" on the COMPAS dataset")
                plt.savefig(Path(savepath+miss+"_"+model+"_"+metric+".png"))
                plt.clf()



In [67]:
a_test = pd.DataFrame(data = {"a": [1,2,3], "b": [np.nan, 1, np.nan]}, columns = ["a","b"])
a_test.isna().sum()

a    0
b    2
dtype: int64

In [84]:
lam['robust_Truepriors_count|gender_factor']['Averaged results']['mcar|acc|rf_cat|fair_reg_90']

[0.6319096710849287, 0.6320078546882669]

In [73]:
for key in lam.keys():
    print(key)
    plotting(lam, key, plot_folders[3]+"-".join(key.split("|")), lambda_imps)

robust_Truepriors_count|gender_factor


100%|██████████| 2/2 [00:08<00:00,  4.40s/it]


robust_Falsepriors_count|gender_factor


100%|██████████| 2/2 [00:10<00:00,  5.39s/it]


<Figure size 2160x1260 with 0 Axes>

In [17]:
for key in compas.keys():
    plotting(compas, key, plot_folders[0]+"-".join(key.split("|")))

  0%|          | 0/2 [00:00<?, ?it/s]findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
findfont: Font family ['normal'] not found. Falling back to DejaVu Sans.
100%|██████████| 2/2 [00:06<00:00,  3.31s/it]


<Figure size 2160x1260 with 0 Axes>

In [25]:
compas['robust_Falsepriors_count|is_Caucasian'].keys()

dict_keys(['Full data', 'Averaged results', 'Standard deviation'])

In [29]:
import plotly.express as px


In [33]:
def plot_acc_vs_fair(df, params, folder_path):
    savepath = folder_path+"/"
    if not os.path.isdir(Path(savepath)):
        os.mkdir(Path(savepath))    
    for miss in tqdm(['mar', 'mcar']):
        for model in ["log_reg", "knn", "rf_cat"]:
            full_data = {NAME_KEYS["acc"]: [],
                         NAME_KEYS["eosum"]:[],
                         NAME_KEYS["acc"]+"_error": [],
                         NAME_KEYS["eosum"]+"_error":[],
                        "Imputation method": []}
            columns = ["Accuracy", NAME_KEYS["eosum"], "Imputation method", NAME_KEYS["acc"]+"_error",  NAME_KEYS["eosum"]+"_error"]
            for imp in IMPUTATIONS:
                key_acc = "|".join([miss,"acc",model,imp])
                key_eosum = "|".join([miss,"eosum",model,imp])
                #print(key_acc,key_eosum)
                data_acc = df[params]["Averaged results"][key_acc]
                acc_error = df[params]['Standard deviation'][key_acc]
                data_eosum = df[params]["Averaged results"][key_eosum]
                eosum_error = df[params]['Standard deviation'][key_eosum]
                full_data["Imputation method"]+=[NAME_KEYS[imp]]#*len(data_acc)
                #print(data_eosum[1])
                full_data[NAME_KEYS["acc"]]+=[data_acc[1]]
                full_data[NAME_KEYS["eosum"]]+=[data_eosum[1]]
                full_data[NAME_KEYS["acc"]+"_error"]+=[acc_error[1]]
                full_data[NAME_KEYS["eosum"]+"_error"]+=[eosum_error[1]]
                #full_data["Missing percentage"].append([NAME_KEYS[imp]]*len(data))
                #print(full_data)
                #print(full_data)
            plotting_data = pd.DataFrame(full_data, columns = columns)
            #print(plotting_data.head())
            fig = plt.gcf()
            fig.set_size_inches(30, 17.5)
            sns.scatterplot(x=NAME_KEYS["acc"], y=NAME_KEYS["eosum"], hue = "Imputation method", data = plotting_data,
                            s = 10000)
            #fig.legend(bbox_to_anchor=(1.2,1))
            plt.title("Performance of "+NAME_KEYS[model]+" on the COMPAS dataset")
            plt.savefig(Path(savepath+miss+"_"+model+"_acc_vs_fair"+".png"))
            plt.clf()
            """fig = px.scatter(plotting_data, x=NAME_KEYS["acc"], y=NAME_KEYS["eosum"], color = "Imputation method",
                 error_x=NAME_KEYS["acc"]+"_error", error_y=NAME_KEYS["eosum"]+"_error")
            fig.show()"""

In [34]:
for key in compas.keys():
    plot_acc_vs_fair(compas, key, plot_folders[0]+"-".join(key.split("|")))

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:02<00:02,  2.36s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:01<00:01,  1.55s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:01<00:01,  1.55s/it]

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:01<00:01,  1.47s/it]

100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


<Figure size 2160x1260 with 0 Axes>

In [21]:
simple.keys()

dict_keys(['x_2|x_1'])

In [35]:
for key in simple.keys():
    plot_acc_vs_fair(simple, key, plot_folders[2]+"-".join(key.split("|")))

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:01<00:01,  1.63s/it]

100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


<Figure size 2160x1260 with 0 Axes>

In [36]:
for key in synth.keys():
    plot_acc_vs_fair(synth, key, plot_folders[1]+"-".join(key.split("|")))

  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:01<00:01,  1.47s/it]

100%|██████████| 2/2 [00:03<00:00,  1.60s/it]


<Figure size 2160x1260 with 0 Axes>